In [69]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf



In [70]:
#  Import and read the charity_data.csv.
import pandas as pd
df = pd.read_csv("/content/all_categories_cleaned_data.csv")
df.head()

,stars,reviews,price,categoryName,isBestSeller,boughtInLastMonth,titleLength
0,4.4,2876,47.69,Industrial Scientific,0,0,60
1,3.8,55,10.99,Industrial Scientific,0,100,152
2,4.0,126,25.99,Industrial Scientific,0,50,160
3,4.5,1936,21.99,Industrial Scientific,0,100,130
4,4.2,46,18.99,Industrial Scientific,0,100,153


In [71]:
# Verify Data Types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1988016 entries, 0 to 1988015
Data columns (total 7 columns):
 #   Column             Dtype  
---  ------             -----  
 0   stars              float64
 1   reviews            int64  
 2   price              float64
 3   categoryName       object 
 4   isBestSeller       int64  
 5   boughtInLastMonth  int64  
 6   titleLength        int64  
dtypes: float64(2), int64(4), object(1)
memory usage: 106.2+ MB


In [72]:
# Determine the number of unique values in each column.
df.nunique()

stars                   42
reviews              20997
price                81372
categoryName           266
isBestSeller             2
boughtInLastMonth       22
titleLength            476
dtype: int64

In [73]:
# Determine the number of unique values in categoryName column.
df['categoryName'].value_counts()

Baby                            24077
Luggage  Travel Gear            22449
Handmade Home Décor             19143
Handmade Kitchen  Dining        19074
Handmade Jewellery              18545
                                ...  
Kids' Ball Pits  Accessories      587
Blu-ray Players  Recorders        467
Smart Home                        434
STEM                              210
International Food Market         152
Name: categoryName, Length: 266, dtype: int64

In [74]:
# Choose a cutoff value of 9000 and create a list of categoryName to be replaced to others
remained_categories = df['categoryName'].value_counts().index[df['categoryName'].value_counts() < 9000]

# Replace in dataframe
for category in remained_categories:
    df['categoryName'] = df['categoryName'].replace(category,"Other")

# Drop rows where categoryName is other
df = df[df['categoryName'] != 'Other']


In [75]:
# Determine the number of unique values in categoryName column.
df['categoryName'].nunique()

53

In [76]:
# Verify rows has been dropped
df['categoryName'].value_counts()

Baby                                     24077
Luggage  Travel Gear                     22449
Handmade Home Décor                      19143
Handmade Kitchen  Dining                 19074
Handmade Jewellery                       18545
Beauty                                   18320
Men's Jewelry                            17969
Women's Watches                          17386
Boys                                     17233
Electronics                              16393
Men's Watches                            16385
Uniforms, Work  Safety                   16150
Men's Shoes                              16049
Women's Shoes                            15964
Beauty Tools  Accessories                15868
Men                                      15768
Shaving  Hair Removal Products           15702
Grocery                                  15601
Women                                    15576
Perfume  Cologne                         15552
Salon  Spa Equipment                     15417
Bath  Body   

In [77]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_df = pd.get_dummies(df)
dummy_df.head()

,stars,reviews,price,isBestSeller,boughtInLastMonth,titleLength,categoryName_Arts Crafts Supplies,categoryName_Automotive Replacement Parts,categoryName_Automotive Tires Wheels,categoryName_Automotive Tools Equipment,...,categoryName_Skin Care Products,categoryName_Snow Ice Sports,"categoryName_Uniforms, Work Safety",categoryName_Women,categoryName_Women's Accessories,categoryName_Women's Clothing,categoryName_Women's Handbags,categoryName_Women's Jewelry,categoryName_Women's Shoes,categoryName_Women's Watches
3355,4.5,43171,29.99,1,0,254,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3356,4.7,215,5.98,1,900,132,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3357,3.9,1006,14.22,1,1000,55,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3358,4.3,17381,12.99,1,900,195,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3359,5.0,1,21.69,1,200,189,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
# Split our preprocessed data into our features and target arrays
y = dummy_df["isBestSeller"].values
X = dummy_df.drop(["isBestSeller"],1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

<ipython-input-78-8524dd1249bb>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = dummy_df.drop(["isBestSeller"],1).values


In [79]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [80]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
no_input_features = len(X_train[0])
hidden_nodes_layer1 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=no_input_features, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 20)                1180      
                                                                 
 dense_5 (Dense)             (None, 1)                 21        
                                                                 
Total params: 1201 (4.69 KB)
Trainable params: 1201 (4.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [81]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [82]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
16872/16872 [==============================] - 26s 1ms/step - loss: 0.0248 - accuracy: 0.9945
Epoch 2/100
16872/16872 [==============================] - 23s 1ms/step - loss: 0.0206 - accuracy: 0.9955
Epoch 3/100
16872/16872 [==============================] - 28s 2ms/step - loss: 0.0203 - accuracy: 0.9956
Epoch 4/100
16872/16872 [==============================] - 22s 1ms/step - loss: 0.0201 - accuracy: 0.9956
Epoch 5/100
16872/16872 [==============================] - 23s 1ms/step - loss: 0.0201 - accuracy: 0.9956
Epoch 6/100
16872/16872 [==============================] - 23s 1ms/step - loss: 0.0200 - accuracy: 0.9956
Epoch 7/100
16872/16872 [==============================] - 23s 1ms/step - loss: 0.0200 - accuracy: 0.9956
Epoch 8/100
16872/16872 [==============================] - 23s 1ms/step - loss: 0.0200 - accuracy: 0.9956
Epoch 9/100
16872/16872 [==============================] - 22s 1ms/step - loss: 0.0199 - accuracy: 0.9956
Epoch 10/100
16872/16872 [====================

In [84]:
X_train_scaled = tf.constant(X_train_scaled)
# Assuming 'nn' is your trained model
# 'X_train_scaled' is your training data input features
# Use the trained model to make predictions on training data
predictions = nn.predict(X_train_scaled)
# Compute gradients of predictions with respect to X_train_scaled
with tf.GradientTape() as tape:
    tape.watch(X_train_scaled)
    predictions = nn(X_train_scaled)
# Compute gradients of predictions with respect to X_train_scaled
gradients = tape.gradient(predictions, X_train_scaled)
# Compute feature importance by taking the absolute mean of gradients
feature_importance = tf.reduce_mean(tf.abs(gradients), axis=0)
# Now 'feature_importance' contains the importance of each feature

16872/16872 [==============================] - 17s 1ms/step


In [91]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

5624/5624 - 5s - loss: 0.0202 - accuracy: 0.9955 - 5s/epoch - 860us/step
Loss: 0.020187977701425552, Accuracy: 0.9955213069915771


In [92]:
# Export our model to HDF5 file
nn.save("test_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
